# Library & I/O + rules of Bridge

## Libary

In [1]:
from sklearn.experimental import enable_hist_gradient_boosting

In [2]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import xgboost as xgb
import time

from sklearn import base, neural_network, neighbors, feature_extraction, datasets, linear_model, metrics, model_selection, preprocessing, svm, ensemble, decomposition, feature_extraction, utils

# Configs
%config InlineBackend.figure_format ='retina'
%matplotlib inline

# Cython
%load_ext Cython
#!python
#cython: language_level=3

print("All libraries set-up and ready to go!")

All libraries set-up and ready to go!


In [3]:
np.set_printoptions(precision=1, edgeitems=10)

In [4]:
from warnings import filterwarnings
filterwarnings('ignore')

## Rules and constants of Bridge

In [5]:
suits = 'cdhs'
strains = ['NT', 's', 'h', 'd', 'c']
levels = '01234567'
card_vals = 'AKQJT98765432'

deck = [c+s for s in suits for c in card_vals]

played_from = ['S', 'E', 'N', 'W']
players = ['N', 'S']

In [6]:
useful_contracts = sorted(['3NT', '4s', '4h', '5d', '5c'] + [l + s for s in strains for l in '67'])

In [7]:
#penalties[0] = NV; penalties[1] = Vul

penalties = [[0, -50, -150, -400, -800, -1100, -1400, -1700, -2000, -2300, -2600, -2900, -3200, -3500],
             [0, -100, -350, -700, -1100, -1400, -1700, -2000, -2300, -2600, -2900, -3200, -3500, -3800]]

base_scores = [[[0, 60, 60, 310, 310, 310, 810, 1310],
               [0, 50, 50, 50, 300, 300, 800, 1300],
               [0, 50, 50, 50, 300, 300, 800, 1300],
               [0, 50, 50, 50, 50, 300, 800, 1300],
               [0, 50, 50, 50, 50, 300, 800, 1300]],
               
               [[0, 60, 60, 510, 510, 510, 1260, 2010],
               [0, 50, 50, 50, 500, 500, 1250, 2000],
               [0, 50, 50, 50, 500, 500, 1250, 2000],
               [0, 50, 50, 50, 50, 500, 1250, 2000],
               [0, 50, 50, 50, 50, 500, 1250, 2000]]]

incr = [30, 30, 30, 20, 20]

In [8]:
def score(contr, made, vul=0.5):
    diff = int(made) - int(contr[0])
    if diff < 0:
        return (vul) * penalties[1][-diff] + (1 - vul) * penalties[0][-diff]
    else:
        str_i = strains.index(contr[1:])
        return made * incr[str_i] + (vul) * base_scores[1][str_i][int(contr[0])] + (1 - vul) * base_scores[0][str_i][int(contr[0])]


In [9]:
score('2NT', -1, 0)

-400

### IMP table

In [27]:
IMPscale = [15, 45, 85, 125, 165, 215, 265, 315, 365, 425, 495, 595, 745, 895, 1095, 1295, 1495, 1745, 1995, 2245, 2495, 2995, 3495, 3995, 99999]

def points_to_IMPs(pt_diff):
    if pt_diff < 0:
        return -points_to_IMPs(-pt_diff)
    for j in range(0, len(IMPscale)):
        if pt_diff < IMPscale[j]:
            return j

In [28]:
def row_eval(idx, contract1, contract2, vul=0.5):
    out1 = []; out2 = []; diff = 0
    s1, s2 = strains.index(contract1[1:]), strains.index(contract2[1:])
    row = pr[idx]
    for tricks in range(0, 14):
        if pr[idx][s1][tricks] > 0:
            out1.append([tricks, int(pr[idx][s1][tricks])])
        if pr[idx][s2][tricks] > 0:
            out2.append([tricks, int(pr[idx][s2][tricks])])
    for i in out1:
        for j in out2:
            diff += i[1] * j[1] * points_to_IMPs(score(contract1, i[0] - 6, vul) - score(contract2, j[0] - 6, vul))
    return diff / (dd_samples ** 2)

### Valid bid sequences

In [12]:
bids = [l+s for l in '123' for s in 'cdhsN'] + ['4c']

In [13]:
in_s = ['(1d)']
sequences = []
for idx in range(len(in_s) + 1, len(bids)):
    new_s = sequences
    for s in sequences + [in_s]:
        if s[-1][1:3] != bids[idx-1]:
            new_s.append(s + [bids[idx-1], "(" + bids[idx] + ")",])
    sequences = new_s

strong_club_seq = [['(1c)'] + s[1:-1] + ['(P)'] for s in sequences if (s[-2] == bids[-2]) or (s[-2] == bids[-3])]

In [14]:
len(strong_club_seq)

377

## I/O

In [15]:
dd_samples = 20

In [16]:
f = open("dNS.txt", "r")
deals = f.read().split("\n")
del deals[-1]
f.close
print("Read ", len(deals), " samples")

Read  632880  samples


In [17]:
d = [{} for _ in range(0, len(deals) // 10)]
for i in range(0, len(deals) // 10):
    d[i]['text'] = deals[10*i][2:18] + ' ' + deals[10*i][36:52]
    
df = pd.DataFrame(data = d)

# Creating hands etc

In [18]:
hands = [{} for _ in range(0, int(len(deals) // 10))]

for i in range(0, int(len(deals) // 10)):
    for pl_id in range(0, len(players)):
        cs = 0; l = []
        for c in deals[10*i][2 + 34*pl_id:18 + 34 * pl_id]:
            if c == '.':
                cs += 1
            else:
                card = c + suits[cs]
                l.append(card)
        hands[i][players[pl_id]] = l

In [19]:
def decodeHex(x):
    if x == 'A':
        return 10.0
    if x == 'B':
        return 11.0
    if x == 'C':
        return 12.0
    if x == 'D':
        return 13.0
    return float(x)

In [20]:
winners = [[] for _ in range(0, int(len(deals) // 10))]

for i in range(0, int(len(deals) // 10)):
    winners[i] = [{} for _ in range(0, len(strains))]
    for strain_idx in range(0, len(strains)):
        winners[i][strain_idx] = {played_from[pl]: [13 - decodeHex(deals[10*i+j][70 + 4*strain_idx + pl]) for j in range(0,10)] for pl in range(0, len(played_from))}

In [21]:
winners[0][0]['N']

[12.0, 12.0, 12.0, 12.0, 12.0, 12.0, 11.0, 12.0, 12.0, 12.0]

# Probabilities of contracts making off DD sample / y engineering

## Computing p_

In [22]:
pr = [[] for _ in range(0, int(len(deals) // 10))]

for i in range(0, int(len(deals) // 10)):
    pr[i] = [[0 for _ in range(0, 14)] for _ in range(0, len(deck))]
    for strain_idx in range(0, len(strains)):
        for n in winners[i][strain_idx]['N'] + winners[i][strain_idx]['S']:
            pr[i][strain_idx][int(n)] += 1 


In [23]:
pr[0][0]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 13, 0]

In [24]:
# old: total point EV

#for j in range(0, 5):
#    for n in range(1, 8):
#        df['ev_' + str(n) + strains[j]] = 0
#        for m in range(0, 8):
#            df['ev_' + str(n) + strains[j]] += df['p_' + str(m) + strains[j]] * score(str(n) + strains[j], m, 0.5) / 20

In [25]:
#useful_cols = ['N', 'S'] + ['p_' + l + s for l in levels for s in strains] + ['ev_' + c for c in useful_contracts]
#df = df[useful_cols]

## Best contract and IMP difference

In [29]:
####### DO NOT RUN #######
## TAKES 1 sec/1000 samples [1 min]

best_c = ['3NT' for _ in range(0, len(df))]

for i in range(0, len(df)):
    bestc = '3NT'
    for c in useful_contracts:
        if row_eval(i, bestc, c) < 0:
            bestc = c
    best_c[i] = bestc

df['best_c'] = best_c

In [30]:
imp_diffs = [[0 for _ in range(0, len(useful_contracts))] for _ in range(0, len(df))]

####### DO NOT RUN #######
## TAKES 1 sec/1000 samples [1 min]

for i in range(0, len(df)):
    bestc = df.at[i, 'best_c']
    for c_idx in range(0, len(useful_contracts)):
        imp_diffs[i][c_idx] = (-1) * row_eval(i, bestc, useful_contracts[c_idx])


In [31]:
imp_diff_3NT = np.array([imp_diffs[i][0] for i in range(0, len(df))])

# Feature eng

In [ ]:
# Takes insanely long to run??

side = [[] for _ in range(0, len(df))]

for i in range(0, len(df)):
    side[i] = [0 for _ in range(0, len(df))]
    for j in range(0, len(deck)):
        for pl in ['N', 'S']:
            if deck[j] in hands[i][pl]:
                side[i][j] = 1

is_NS = np.array(side)

In [ ]:
is_NS[0]

# Running and comparing models

## Wrapper

In [ ]:
def train_eval(models,
               features=[(is_NS, '52-vector with 0/1 if it belongs to NS')], 
               targets=[(imp_diff_3NT, 'IMP-difference for playing 3NT vs best c')]):
    
    if not isinstance(models, list):
        models = [models]
    
    for X, f_desc in features:
        print ("Features: " + f_desc + "\n")
        for y, t_desc in targets:
            print ("    Targets: " + t_desc)
            ts = time.time(); accuracies = []
            X_tr, X_ts, y_tr, y_ts = model_selection.train_test_split(X, y, test_size=0.2, random_state=1)
            
            for m_d in models:
                if 'desc' not in m_d:
                    m_d['desc'] = 'unknown m '
                for kw in 'bfp':
                    if kw+'_args' not in m_d:
                        m_d[kw+'_args'] = {}
                
                if isinstance(m_d['m'], base.BaseEstimator):
                    m = m_d['m']
                else:
                    # Model has to be built
                    m = m_d['m'](input_s=X_tr.shape[1], **m_d['b_args'])
                
                ts = time.time()
                m.fit(X_tr, y_tr, **m_d['f_args'])
                print ("        M: " + m_d['desc'] + " # ", end='')
                y_pred = m.predict(X_ts, **m_d['p_args'])
                y_tr_pred = m.predict(X_tr, **m_d['p_args'])
                #print ("P:",  y_pred, end='')
                print (" test MAE: " + "{0:.2f}".format(metrics.mean_absolute_error(y_ts, y_pred)), end='' )
                print (" - tr MAE: " + "{0:.2f}".format(metrics.mean_absolute_error(y_tr, y_tr_pred)), end='' )
                print (" T: " + "{0:.1f}".format(time.time() - ts) + "s")
                